<a href="https://colab.research.google.com/github/RohitKarake/ecg_preprocessing/blob/main/denoise_raw_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -r -N -c -np https://physionet.org/files/mitdb/1.0.0/
!pip install wfdb
!pip install neurokit2

Streaming output truncated to the last 5000 lines.
2020-10-16 08:11:34 (5.58 MB/s) - ‘physionet.org/files/mitdb/1.0.0/228.dat’ saved [1950000/1950000]

--2020-10-16 08:11:34--  https://physionet.org/files/mitdb/1.0.0/228.hea
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 302 [text/plain]
Saving to: ‘physionet.org/files/mitdb/1.0.0/228.hea’

physionet.org/files 100%[===================>]     302  --.-KB/s    in 0s      

2020-10-16 08:11:34 (50.8 MB/s) - ‘physionet.org/files/mitdb/1.0.0/228.hea’ saved [302/302]

--2020-10-16 08:11:34--  https://physionet.org/files/mitdb/1.0.0/228.xws
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 88 [text/plain]
Saving to: ‘physionet.org/files/mitdb/1.0.0/228.xws’

physionet.org/files 100%[===================>]      88  --.-KB/s    in 0s      

2020-10-16 08:11:34 (15.9 MB/s) - ‘physionet.org/files/mitdb/1.0.0/228.xws’ saved [88/88]

--202

In [4]:
import wfdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import neurokit2 as nk
import numpy as np
import scipy.signal as signal
import pywt
from matplotlib.lines import Line2D
import plotly.graph_objects as go


In [5]:

ORIGINAL_SAMPLING_RATE = 5000
lower_band = 0.5
upper_band = 60

# BANDPASS FILTER REFERENCE https://www.sciencedirect.com/science/article/abs/pii/S0167865519302004
bandpass_filter = signal.butter(4, [lower_band, upper_band], btype='bandpass', output='sos', fs = ORIGINAL_SAMPLING_RATE)
DIR_LOC = './physionet.org/files/mitdb/1.0.0/'
FILE_LOC = DIR_LOC + '100'
sample = wfdb.rdsamp(FILE_LOC)
signals = sample[0]
raw_ecg_signal = signals[:,0]
print(raw_ecg_signal[:15])
signals.shape

[-0.145 -0.145 -0.145 -0.145 -0.145 -0.145 -0.145 -0.145 -0.12  -0.135
 -0.145 -0.15  -0.16  -0.155 -0.16 ]


(650000, 2)

In [ ]:
### BAND FILTERING
filtered_signal = signal.sosfiltfilt(bandpass_filter, raw_ecg_signal)
print(filtered_signal[:15])

In [ ]:
### DOWNSAMPLING FROM 5000Hz to 500Hz
downsampled_ecg_signal = signal.decimate(filtered_signal, 10)
print(downsampled_ecg_signal[:15])

In [ ]:
### SMOOTHING SIGNAL
smoothened_ecg_signal = signal.savgol_filter(downsampled_ecg_signal, window_length = 17, polyorder = 3)
print(smoothened_ecg_signal[:15])

In [ ]:
f, t, Sxx = signal.spectrogram(raw_ecg_signal, ORIGINAL_SAMPLING_RATE)
plt.pcolormesh(t, f, np.log10(Sxx), shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
lower_band2 = 0.3
upper_band2 = 5
bandpass_filter2 = signal.butter(4, [lower_band2, upper_band2], btype='bandpass', output='sos', fs = ORIGINAL_SAMPLING_RATE)
filtered_signal2 = signal.sosfiltfilt(bandpass_filter2, raw_ecg_signal)
print(filtered_signal2[:15])

In [ ]:
ecg2 = signal.sosfiltfilt(bandpass_filter2, raw_ecg_signal)
plt.plot(ecg2[:5000])

In [ ]:
clean_signal = pd.DataFrame({"ECG_Raw" : raw_ecg_signal,
...                          "ECG_NeuroKit" : nk.ecg_clean(raw_ecg_signal, sampling_rate=5000, method="neurokit")})

clean_signal

In [ ]:
plt.plot(clean_signal['ECG_Raw'][:1000])